In [ ]:
from utils.PinConfig import PinConfig
import casadi
import pinocchio as pin

config_file = 'config/SimpleConfig.yaml'
robot, model, data, cmodel, cdata = PinConfig(config_file)
print(model)

cq = casadi.SX.sym('q', model.nq)

In [ ]:
from utils.ObstacleSpheres import *
import numpy as np
config_file = 'config/Config.yaml'
obs = ObsConfig(config_file)
print(obs.robot_config)

obs.spheres_from_robot(q0= [0.0,0,0,0,0,0])  # Get initial robot obstacle spheres
#print(obs.robot_sphere_list)
get_obstacle_distances = casadi.Function(
    'get_obstacle_distances', 
    [cq], 
    [obstacle_distances(obs.env_spheres, obs.robot_spheres, cmodel, cdata, cq)]
    )

In [ ]:
nq = model.nq   # Number of joints
nv = model.nv   # Number of DOF
nx = nq + nv    # Number of states
cx = casadi.SX.sym("x", nx, 1)
ctau = casadi.SX.sym("tau", nv, 1)
cacc = cpin.aba(cmodel, cdata, cx[:nq], cx[nq:], ctau) # Forward dynamics

cq = casadi.SX.sym("x", model.nq, 1)


# Joint limits
lowerVelocityLimit = np.pi/2 * np.array([-1, -1, -1, -1, -1, -1])  # rad/s
upperVelocityLimit = np.pi/2 * np.array([1, 1, 1, 1, 1, 1])
lowerTorqueLimit = 100 * np.array([-1, -1, -1, -1, -1, -1]) 
upperTorqueLimit = 100 * np.array([1, 1, 1, 1, 1, 1])
print(model.upperPositionLimit)
print(model.lowerPositionLimit)

waypoints = np.array([[0, np.pi/4, 0, 0, 0, 0], 
                      [-np.pi/6, np.pi/3, np.pi/6, -np.pi/6, -np.pi/3, -np.pi/4],
                      [-np.pi/4, np.pi/4, np.pi/3, 0, -np.pi/6, -np.pi/2]])
q0 = waypoints[0]
robot.q0 = q0
qT = waypoints[-1]
tau0 = pin.rnea(model, data, q0, np.zeros(nv), np.zeros(nv))
tauT = pin.rnea(model, data, qT, np.zeros(nv), np.zeros(nv))
print(tau0)
print(tauT)

In [ ]:
from utils.Lagrange import *
from utils.LGL_collocation import LGL_collocation
from utils.LocalChart import *

Ntp = 10
Np = waypoints.shape[0]-1  # Number of phase (dersired waypoints: N+1)
T = Np*Ntp  # Number of total collocation points / optimization variables

LGL_Pts, LGL_Wts = Normalize_LGL_Pts_Wts(Ntp)
DL,_ = differentiation_matrix(Ntp, LGL_Pts)

LGL = LGL_collocation(Ntp, nq, nv)
StatesDiff, StateDiff, LocalsDiff, LocalDiff= LGL.getDiffFuncs()

Feq = casadi.Function(
    "Dyn",
    [cx, ctau],
    [casadi.vertcat(cx[nq:], cacc)],
)

In [ ]:
opti = casadi.Opti()
# Optimization variables
var_xs = [opti.variable(nx) for _ in range(T)]
var_us = [opti.variable(nv) for _ in range(T)]
var_deltaT = [opti.variable(1) for _ in range(Np)]

# Constraints
for i in range(Np):
    opti.subject_to(var_deltaT[i] >= 0.0)

    for tp in range(Ntp):
        t = i*Ntp + tp
        # dynamics
        opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[tp,:]) 
                        == Feq(var_xs[t], var_us[t]))
        
        # joint constraints
        opti.subject_to(var_xs[t][:nq] <= model.upperPositionLimit)
        opti.subject_to(var_xs[t][:nq] >= model.lowerPositionLimit)
        # joint velocity constraints
        opti.subject_to(var_xs[t][nq:] <= upperVelocityLimit)
        opti.subject_to(var_xs[t][nq:] >= lowerVelocityLimit)
        # torque constraints
        opti.subject_to(var_us[t] <= upperTorqueLimit)
        opti.subject_to(var_us[t] >= lowerTorqueLimit)

        # collision avoidance
        opti.subject_to(get_obstacle_distances(var_xs[t][:nq]) >= 0.05)

# waypoints constraints (N-1 middle waypoints)
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp][:nq] == waypoints[i])

# continuity
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp-1] == var_xs[i*Ntp])
    opti.subject_to(var_us[i*Ntp-1] == var_us[i*Ntp])

# continuity of derivatives
for i in range(Np-1):
    opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:]) 
                    == StatesDiff(casadi.vertcat(*var_xs[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i+1], DL[0,:]))
    
    opti.subject_to(LocalDiff(casadi.vertcat(*var_us[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:])
                   == LocalDiff(casadi.vertcat(*var_us[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i], DL[0,:]))


# boundary conditions
opti.subject_to(var_xs[0][:nq] == q0)
opti.subject_to(var_xs[-1][:nq] == qT)
opti.subject_to(var_xs[0][nq:] == 0)
opti.subject_to(var_xs[-1][nq:] == 0)
#opti.subject_to(var_us[0] == tau0)
#opti.subject_to(var_us[-1] == tauT)

# Objective
totalcost = 0

R = np.eye(nv)
for i in range(Np):
    for tp in range(Ntp):
        var_xs_Phase = var_xs[i*Ntp: (i+1)*Ntp]
        dq = [dq_Phase[nq:] for dq_Phase in var_xs_Phase]
        ddq = StateDiff(casadi.vertcat(*dq), var_deltaT[i], DL[tp,:])
        totalcost += var_deltaT[i]/2 * LGL_Wts[tp] * casadi.mtimes([ddq.T, R, ddq])
        
    totalcost += var_deltaT[i]

opti.minimize(totalcost)
opti.solver("ipopt") 

for x in var_xs:
    opti.set_initial(x, np.concatenate([robot.q0, np.zeros(nv)]))
for u in var_us:
    opti.set_initial(u, tau0)
for t in var_deltaT:
    opti.set_initial(t, 1)


try:
    sol = opti.solve_limited()
    sol_xs = [opti.value(var_x) for var_x in var_xs]
    sol_us = [opti.value(var_u) for var_u in var_us]
    sol_T = [opti.value(var_t) for var_t in var_deltaT]
except:
    print("ERROR in convergence, plotting debug info.")
    sol_xs = [opti.debug.value(var_x) for var_x in var_xs]
    sol_us = [opti.debug.value(var_u) for var_u in var_us]
